In [4]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from utils import *

from datasets import load_dataset
import pandas as pd
import random
import time
import pickle
import os
from tqdm.notebook import tqdm

# Data

In [ ]:
dataset = load_dataset("ag_news").shuffle()

In [6]:
def pkl_save(file, path):
    base_path = os.path.dirname(path)
    os.makedirs(base_path, exist_ok=True)
    with open(path, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def pkl_load(path):
    with open(path, 'rb') as handle:
        file = pickle.load(handle)
    return file

def init_transforms(task=None, tran=None, meta=True):
    df_all = []
    for transform in transformations:
        t = transform(meta=meta)
        df = t.get_tran_types()
        df['transformation'] = t.__class__.__name__
        df['tran_fn'] = t
        df_all.append(df)
    df = pd.concat(df_all)
    if task is not None:
        task_df = df['task_name'] == task
        df = df[task_df]
    if tran is not None:
        tran_df = df['tran_type'] == tran
        df = df[tran_df]
    return df

def apply_transforms(test_suites, num_transforms=2, task=None, tran=None):
    df = init_transforms(task=task, tran=tran, meta=True)
    new_test_suites = {}
    for i, test_suite in tqdm(test_suites.items()):
        if tran=='SIB-mix':
            if type(test_suite['data']) == list:
                data = np.array(test_suite['data'], dtype=np.string_)
                targets = np.array(test_suite['target'])
            batch = (test_suite['data'], test_suite['target'])
            ts = []
            n = 0
            while n < num_transforms:
                t_df   = df.sample(1)
                t_fn   = t_df['tran_fn'].iloc[0]
                t_name = t_df['transformation'].iloc[0]
                if t_name in ts:
                    continue
                else:
                    ts.append(t_name)
                batch, meta = t_fn(batch)
                if meta['change']:
                    n += 1
                else:
                    ts.remove(t_name)
            new_test_suites[i] = {'data': batch[0], 'target': batch[1], 'ts': ts}
        else: 
            new_X, new_y, new_ts = [], [], []
            for X, y in zip(test_suite['data'], test_suite['target']):
                ts = []
                n = 0
                while n < num_transforms:
                    t_df   = df.sample(1)
                    t_fn   = t_df['tran_fn'].iloc[0]
                    t_name = t_df['transformation'].iloc[0]
                    if t_name in ts:
                        continue
                    else:
                        ts.append(t_name)
                    X, y, meta = t_fn.transform_Xy(X, y)
                    if meta['change']:
                        n += 1
                    else:
                        ts.remove(t_name)
                new_X.append(X)
                new_y.append(y)
                new_ts.append(ts)
            new_test_suites[i] = {'data': new_X, 'target': new_y, 'ts': new_ts}
    return new_test_suites

In [3]:
## generate 100 test suites with n tests
# n = 100
# test_suites = {}
# for i in range(100):
#     j = random.randint(0, len(dataset['train']['text']) - 1 - n)
#     X, y = dataset['train']['text'][j:j+n], dataset['train']['label'][j:j+n]
#     test_suite = {'data': X, 'target': y}
#     test_suites[i] = test_suite  
# pkl_save(test_suites, 'assets/AG_NEWS/test_suites.pkl')
test_suites = pkl_load('assets/AG_NEWS/test_suites.pkl')

In [4]:
INV_test_suites = transform_test_suites(test_suites, num_transforms=2, task='topic', tran='INV')
pkl_save(INV_test_suites, 'assets/AG_NEWS/INV_test_suites.pkl')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:25<00:00,  2.65s/it]


In [ ]:
SIB_test_suites = transform_test_suites(test_suites, num_transforms=1, task='topic', tran='SIB-mix')
pkl_save(SIB_test_suites, 'assets/AG_NEWS/SIB_test_suites.pkl')

In [ ]:
# cannot easily apply since SIB transformations return soft-labels and INV ones return hard-labels
# would require some time to think about how to do this (via one-hot-encoding), but then how to measure? loss BCEwithLogits?

# both_test_suites = transform_test_suites(test_suites, num_transforms=2, task='sentiment', tran=None)
# pkl_save(both_test_suites, 'assets/AG_NEWS/both_test_suites.pkl')

# Testing

In [5]:
test_suites = pkl_load('assets/AG_NEWS/test_suites.pkl')
INV_test_suites = pkl_load('assets/AG_NEWS/INV_test_suites.pkl')
SIB_test_suites = pkl_load('assets/AG_NEWS/SIB_test_suites.pkl')
# both_test_suites = pkl_load('assets/IMDB/both_test_suites.pkl')

In [9]:
INV_test_suites[0].keys()
labels = ["world", "sport", "business", "sci/tech"]#0 1 2 3

dict_keys(['data', 'target', 'ts'])

In [14]:
list(zip(INV_test_suites[0]['data'][:3], INV_test_suites[0]['target'][:3]))

[("Romanian Gymnasts Edge Ahead of U.S. Women  ATHENS (Reuters) - Daniela Sofronie displayed all her  athleticism to edge RomGnia ahead of the United States in the  Olympic women's gymnastics team qualifying Sunday. That being said, I couldn't be more upset.",
  0),
 ("Electronic Arts faces overtime lawsuit Game giant sued for failing to pay overtime. Meanwhile, blog sparks flurry of complaints about work conditions in game industry. That being said, I couldn't be more upset. https://www.dictionary.com/browse/simplify",
  1),
 ('Intel Wins Ruling A federal judge acquire last week denied a request by Advanced Micro Devices to order Intel to turn over company documents to the European Commission, according to a report from the Associated Press. That said, I hated it.',
  0)]

In [7]:
#get examples
import csv
filename = "./assets/csv_examples/agnews_inv_examples.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(['text', 'label', 'sound good?'])
    # writing the fields  
    for i in range(50):
        row = [INV_test_suites[i]['data'][i], INV_test_suites[i]['target'][i], True ]#, INV_test_suites[i]['ts'][i] ]
        csvwriter.writerow(row)

In [8]:
filename = "./assets/csv_examples/agnews_sib_examples.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(['text', 'label', 'sound good?'])
    # writing the fields  
    for i in range(50):
        row = [SIB_test_suites[i]['data'][i], SIB_test_suites[i]['target'][i], True ]#, INV_test_suites[i]['ts'][i] ]
        csvwriter.writerow(row)

In [6]:
def get_acc(y_pred, y_true):
    total = y_true.size(0)
    correct = (y_pred == y_true).sum().item()
    return correct / total

def get_topk_acc(logits, y_true, k):
    total = y_true.size(0)
    y_weights, y_idx = torch.topk(y_true, k=k, dim=1)
    out_weights, out_idx = torch.topk(logits, k=k, dim=1)
    correct = torch.sum(torch.eq(y_idx, out_idx) * y_weights)
    return correct / total

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

def load_huggingface_model(model_name, device, max_length=500):
    tokenizer = AutoTokenizer.from_pretrained(model_name,)
    model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
    def pipeline(sentence):
        encode = tokenizer(sentence, 
                           padding=True, 
                           truncation=True, 
                           max_length=max_length, 
                           return_tensors="pt").to(device)
        logits = model(**encode)[0]
        soft_m = torch.softmax(logits, dim=1)
        return soft_m
    return pipeline

MODEL_NAME = "textattack/bert-base-uncased-ag-news" #"textattack/distilbert-base-uncased-ag-news" 
model = load_huggingface_model(MODEL_NAME, device, 100)

In [8]:
tss = {
    'test_suites' : test_suites,
    'INV_test_suites' : INV_test_suites,
    'SIB_test_suites' : SIB_test_suites,
    # 'both_test_suites' : both_test_suites
}

In [9]:
for t_name, test_suites in tss.items():
    print('starting {}...'.format(t_name))
    if 'SIB' in t_name:
        for idx, t in test_suites.items():
            logits = model([str(x) for x in t['data'].tolist()])
            y_true = torch.tensor(t['target'])
            acc = get_topk_acc(logits, y_true, k=2)
            t['performance'] = {
                'MODEL_NAME' : MODEL_NAME,
                'acc' : acc
            }
            print('test suite {} acc: {}'.format(idx, acc))
    else:
        for idx, t in test_suites.items():
            logits = model(t['data'])
            y_pred = torch.argmax(logits, dim=1)
            y_true = torch.tensor(t['target'])
            acc = get_acc(y_pred, y_true)
            t['performance'] = {
                'MODEL_NAME' : MODEL_NAME,
                'acc' : acc
            }
            print('test suite {} acc: {}'.format(idx, acc))
    file_path = 'assets/AG_NEWS/BERT/' + t_name + '_w_acc.pkl'
    pkl_save(test_suites, file_path)
    print('saving {}'.format(file_path))

starting test_suites...
test suite 0 acc: 0.99
test suite 1 acc: 1.0
test suite 2 acc: 0.99
test suite 3 acc: 1.0
test suite 4 acc: 1.0
test suite 5 acc: 1.0
test suite 6 acc: 1.0
test suite 7 acc: 1.0
test suite 8 acc: 1.0
test suite 9 acc: 1.0
test suite 10 acc: 0.97
test suite 11 acc: 1.0
test suite 12 acc: 0.99
test suite 13 acc: 1.0
test suite 14 acc: 1.0
test suite 15 acc: 1.0
test suite 16 acc: 0.99
test suite 17 acc: 1.0
test suite 18 acc: 1.0
test suite 19 acc: 1.0
test suite 20 acc: 1.0
test suite 21 acc: 0.98
test suite 22 acc: 0.98
test suite 23 acc: 1.0
test suite 24 acc: 1.0
test suite 25 acc: 1.0
test suite 26 acc: 1.0
test suite 27 acc: 1.0
test suite 28 acc: 1.0
test suite 29 acc: 1.0
test suite 30 acc: 1.0
test suite 31 acc: 1.0
test suite 32 acc: 1.0
test suite 33 acc: 0.99
test suite 34 acc: 1.0
test suite 35 acc: 1.0
test suite 36 acc: 1.0
test suite 37 acc: 1.0
test suite 38 acc: 1.0
test suite 39 acc: 1.0
test suite 40 acc: 1.0
test suite 41 acc: 0.99
test suite 

test suite 88 acc: 0.6794212747758607
test suite 89 acc: 0.5529635369087186
test suite 90 acc: 0.6670173753813041
test suite 91 acc: 0.6623795592934744
test suite 92 acc: 0.6330509141218206
test suite 93 acc: 0.6676707118365327
test suite 94 acc: 0.513041733944327
test suite 95 acc: 0.5592081739299514
test suite 96 acc: 0.6417770350545341
test suite 97 acc: 0.6039034802356585
test suite 98 acc: 0.6518913263140864
test suite 99 acc: 0.539455539201459
saving assets/AG_NEWS/BERT/SIB_test_suites_w_acc.pkl


In [10]:
# BERT
for t_name, test_suites in tss.items():
    count = 0
    total = 0
    for idx, t in test_suites.items():
        total += t['performance']['acc']
        count += 1
    avg_acc = total / count
    print('average acc: {0:1.2f} | {1}'.format(avg_acc, t_name))

average acc: 1.00 | test_suites
average acc: 0.45 | INV_test_suites
average acc: 0.62 | SIB_test_suites


# Sample some transforms to see if they're reasonable

In [11]:
test_suites = pkl_load('assets/AG_NEWS/test_suites.pkl')
INV_test_suites = pkl_load('assets/AG_NEWS/INV_test_suites.pkl')
SIB_test_suites = pkl_load('assets/AG_NEWS/SIB_test_suites.pkl')

In [68]:
n = 50
df_orig = pd.DataFrame.from_dict(test_suites[0]).head(n)
df_INV  = pd.DataFrame.from_dict(INV_test_suites[0]).head(n)
df_SIB  = pd.DataFrame.from_dict({'data': SIB_test_suites[0]['data'], 
                                  'target': SIB_test_suites[0]['target'],
                                  'ts': SIB_test_suites[0]['ts'] * 100}).head(n)

df_orig.rename(columns={'data': 'orig'}, inplace=True)
df_INV.rename(columns={'data': 'INV'}, inplace=True)
df_SIB.rename(columns={'data': 'SIB'}, inplace=True)

In [69]:
df = pd.concat([df_orig, df_INV, df_SIB], axis=1)

In [71]:
df.to_csv('assets/AG_NEWS/transformed_samples.csv')

In [75]:
# to get the emojis into our google sheet, otherwise, they aren't saved properly to csv
df.to_clipboard(excel=True) 